In [46]:
import torch
from torch.distributions import Normal, Laplace
from testarchitectures import get_stock_transforms
from zuko.flows import UnconditionalDistribution
from causalflows.flows import CausalFlow
from causal_cocycle.conditionalflow_helper import sample_do, sample_cf
from torch.distributions import MultivariateNormal, Independent

In [61]:
torch.manual_seed(0)
# Context X: 10 rows, single context dimension
N = 10
X = torch.randn(N, 1)
# Feature Y will be output dim of flow; set y_dim=1
y_dim = 2

# Build a trivial transform for test (e.g., MAF or affine)
transforms = get_stock_transforms(x_dim=1, y_dim=y_dim, mask=None)
transform = transforms[0]

# Base distribution: 1-D event shape
#base = UnconditionalDistribution(
#    Normal,
#    loc=torch.zeros(1,y_dim),
#    scale=torch.ones(1,y_dim),
#    buffer=True
#)


base = UnconditionalDistribution(
    MultivariateNormal,
    loc=torch.zeros(y_dim),
    covariance_matrix=torch.eye(y_dim),
    buffer=True
)

def base_fn(c):
    # ignore context c for unconditional base
    loc = torch.zeros(y_dim, device=c.device)
    scale = torch.ones(y_dim, device=c.device)
    return Independent(Laplace(loc, scale), reinterpreted_batch_ndims=1)

base = base_fn

# Construct causal flow
flow = CausalFlow(transform=transform, base=base)

flow(X)

CausalNormalizingFlow(
  (transform): AutoregressiveTransform()
  (base): Independent(Laplace(loc: torch.Size([10, 2]), scale: torch.Size([10, 2])), 1)
)

In [69]:
transforms[0](X)(torch.ones((N,y_dim)))

tensor([[1.3397, 0.9906],
        [1.5576, 0.8785],
        [1.7817, 0.7633],
        [1.4552, 0.9312],
        [1.6516, 0.8302],
        [1.6890, 0.8110],
        [1.4749, 0.9211],
        [1.4232, 0.9476],
        [1.6082, 0.8525],
        [1.5707, 0.8718]], grad_fn=<AddBackward0>)